In [1]:
import numpy as np
import pandas as pd

In [3]:
schedule = pd.read_csv("https://raw.githubusercontent.com/njj06135/Baseball_ChilliShrimp/master/data/schedule.csv", encoding = 'CP949')
pa_b = pd.read_csv("https://raw.githubusercontent.com/njj06135/Baseball_ChilliShrimp/master/data/batter_predict_pa.csv")
modeling_b = pd.read_csv("https://raw.githubusercontent.com/njj06135/Baseball_ChilliShrimp/master/data/batter_feature_selection.csv")

### batter

In [4]:
batter = modeling_b.loc[(modeling_b['year']==2020) & (modeling_b['month']>7)]

In [5]:
batter = batter[['T_ID', 'P_ID']].drop_duplicates()

In [6]:
pa_b['P_ID']=pa_b['P_ID'].astype('int')

In [7]:
batter_left= pd.merge(pa_b, batter, left_on='P_ID', right_on='P_ID', how='left')
batter_left

,year,month,P_ID,PA,T_ID
0,2020,10,65304,3.0,WO
1,2020,10,64012,10.0,KT
2,2020,10,68204,4.0,OB
3,2020,10,66203,6.0,OB
4,2020,10,50293,2.0,OB
...,...,...,...,...,...
222,2020,10,63435,4.0,SS
223,2020,10,69418,0.0,SS
224,2020,10,61457,0.0,SS
225,2020,10,66707,7.0,HH


In [8]:
batter_tidy = pd.read_csv("https://raw.githubusercontent.com/njj06135/Baseball_ChilliShrimp/master/data/batter_tidy.csv")

In [9]:
babip_total = batter_tidy.groupby(['P_ID']).sum()

In [10]:
babip_total['BABIP'] = (babip_total['HIT'] - babip_total['HR']) / (babip_total['AB'] - babip_total['KK'] - babip_total['HR'] + babip_total['SF'])

In [11]:
babip_total = babip_total[['BABIP']]

In [12]:
babip_total = babip_total.fillna(0)

In [13]:
babip_total

,BABIP
P_ID,
50054,0.250000
50066,0.277778
50150,0.434783
50165,0.331950
50202,0.240000
...,...
99445,0.000000
99563,0.000000
99606,0.362687


In [14]:
batter_2020 = batter_tidy.loc[batter_tidy.year == 2020, :]
batter_2020 = batter_2020.groupby(['P_ID']).sum()

In [15]:
batter_2020['BABIP_2020'] = (batter_2020['HIT'] - batter_2020['HR']) / (batter_2020['AB'] - batter_2020['KK'] - batter_2020['HR'] + batter_2020['SF'])

In [16]:
babip_2020 = batter_2020[['BABIP_2020']]

In [17]:
babip_total['P_ID'] = babip_total.index
babip_total = babip_total.reset_index(drop = True)

In [18]:
babip_2020['P_ID'] = babip_2020.index

C:\Users\dhxog\anaconda3\envs\my\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [19]:
babip_2020 = babip_2020.reset_index(drop = True)

In [20]:
batter_left= pd.merge(batter_left, babip_total, left_on='P_ID', right_on='P_ID', how='left')
batter_left= pd.merge(batter_left, babip_2020, left_on='P_ID', right_on='P_ID', how='left')


In [23]:
batter_left = batter_left.fillna(0)

In [24]:
batter_left['total_BABIP']=batter_left['BABIP_2020']-(batter_left['BABIP_2020']-batter_left['BABIP'])/5
batter_left

,year,month,P_ID,PA,T_ID,BABIP,BABIP_2020,total_BABIP
0,2020,10,65304,3.0,WO,0.375000,1.000000,0.875000
1,2020,10,64012,10.0,KT,0.297619,0.227273,0.241342
2,2020,10,68204,4.0,OB,0.315789,0.315789,0.315789
3,2020,10,66203,6.0,OB,0.285714,0.352941,0.339496
4,2020,10,50293,2.0,OB,0.266667,0.266667,0.266667
...,...,...,...,...,...,...,...,...
222,2020,10,63435,4.0,SS,0.166667,0.166667,0.166667
223,2020,10,69418,0.0,SS,0.299363,0.300699,0.300432
224,2020,10,61457,0.0,SS,0.000000,0.000000,0.000000
225,2020,10,66707,7.0,HH,0.272727,0.285714,0.283117


In [25]:
df1 = batter_tidy.copy()
df1.head()

,year,month,P_ID,T_ID,AB,HIT,H2,H3,HR,BB,...,VS_T_ID_KT,VS_T_ID_LG,VS_T_ID_LT,VS_T_ID_NC,VS_T_ID_OB,VS_T_ID_SK,VS_T_ID_SS,VS_T_ID_WO,HOME,AWAY
0,2016,4,60146,LG,0,0,0,0,0,0,...,2,0,3,3,0,3,2,3,10,12
1,2016,4,60263,NC,0,0,0,0,0,0,...,0,3,4,0,3,3,3,2,9,14
2,2016,4,60404,HH,12,2,0,0,0,0,...,0,4,3,3,6,0,2,3,12,11
3,2016,4,60456,SS,65,15,3,1,0,5,...,6,2,3,3,4,0,0,0,10,13
4,2016,4,60523,LT,91,22,4,1,0,7,...,3,3,0,4,0,3,3,3,14,11


In [26]:
df1 = df1.loc[df1['year']==2020]

In [27]:
df1

,year,month,P_ID,T_ID,AB,HIT,H2,H3,HR,BB,...,VS_T_ID_KT,VS_T_ID_LG,VS_T_ID_LT,VS_T_ID_NC,VS_T_ID_OB,VS_T_ID_SK,VS_T_ID_SS,VS_T_ID_WO,HOME,AWAY
6273,2020,5,50066,KT,16,4,2,0,0,1,...,0,3,3,3,2,0,3,3,12,11
6274,2020,5,50150,LG,0,0,0,0,0,0,...,3,0,0,2,3,3,3,2,11,11
6275,2020,5,50165,LG,80,30,5,0,10,10,...,3,0,0,2,3,3,3,2,11,11
6276,2020,5,50202,OB,14,3,0,0,0,1,...,2,3,6,3,0,3,3,0,11,12
6277,2020,5,50300,WO,35,4,0,0,1,1,...,3,2,3,3,0,3,3,0,12,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7419,2020,9,63435,SS,4,0,0,0,0,0,...,4,2,2,3,4,2,0,2,13,11
7420,2020,9,69418,SS,0,0,0,0,0,0,...,4,2,2,3,4,2,0,2,13,11
7421,2020,9,61457,SS,0,0,0,0,0,0,...,4,2,2,3,4,2,0,2,13,11
7422,2020,9,66707,HH,7,0,0,0,0,0,...,2,2,2,2,5,2,2,4,16,9


In [28]:
df1 = df1.groupby(['P_ID']).sum()
df1['inplay_ratio'] = (df1['AB']-df1['KK']-df1['HR']+df1['SF'])/df1['PA']

In [29]:
df1

,year,month,AB,HIT,H2,H3,HR,BB,HP,KK,...,VS_T_ID_LG,VS_T_ID_LT,VS_T_ID_NC,VS_T_ID_OB,VS_T_ID_SK,VS_T_ID_SS,VS_T_ID_WO,HOME,AWAY,inplay_ratio
P_ID,,,,,,,,,,,,,,,,,,,,,
50054,8080,30,65,13,3,0,0,7,1,13,...,8,9,12,10,12,10,10,45,47,0.712329
50066,6060,18,25,6,2,0,1,2,0,6,...,7,9,11,5,4,7,6,36,32,0.666667
50150,10100,35,29,10,2,0,0,1,0,6,...,0,12,11,15,14,13,14,56,61,0.766667
50165,10100,35,399,114,17,2,34,51,3,128,...,0,12,11,15,14,13,14,56,61,0.517167
50202,8080,26,28,6,0,0,0,1,0,4,...,13,14,14,0,10,12,6,47,46,0.833333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79456,10100,35,223,68,9,2,0,15,4,25,...,14,16,13,11,13,16,0,65,59,0.808000
79545,2020,6,0,0,0,0,0,0,0,0,...,3,0,1,0,0,3,3,10,13,NaN
79608,10100,35,371,106,26,0,6,32,7,44,...,12,0,10,14,12,13,16,56,60,0.778571


In [30]:
df1['P_ID'] = df1.index

In [31]:
df1 = df1.reset_index(drop = True)

In [32]:
df1

,year,month,AB,HIT,H2,H3,HR,BB,HP,KK,...,VS_T_ID_LT,VS_T_ID_NC,VS_T_ID_OB,VS_T_ID_SK,VS_T_ID_SS,VS_T_ID_WO,HOME,AWAY,inplay_ratio,P_ID
0,8080,30,65,13,3,0,0,7,1,13,...,9,12,10,12,10,10,45,47,0.712329,50054
1,6060,18,25,6,2,0,1,2,0,6,...,9,11,5,4,7,6,36,32,0.666667,50066
2,10100,35,29,10,2,0,0,1,0,6,...,12,11,15,14,13,14,56,61,0.766667,50150
3,10100,35,399,114,17,2,34,51,3,128,...,12,11,15,14,13,14,56,61,0.517167,50165
4,8080,26,28,6,0,0,0,1,0,4,...,14,14,0,10,12,6,47,46,0.833333,50202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349,10100,35,223,68,9,2,0,15,4,25,...,16,13,11,13,16,0,65,59,0.808000,79456
350,2020,6,0,0,0,0,0,0,0,0,...,0,1,0,0,3,3,10,13,NaN,79545
351,10100,35,371,106,26,0,6,32,7,44,...,0,10,14,12,13,16,56,60,0.778571,79608
352,4040,11,18,3,1,0,0,3,1,6,...,6,9,2,6,3,6,23,24,0.600000,79705


In [33]:
df1.isna().sum()

year             0
month            0
AB               0
HIT              0
H2               0
H3               0
HR               0
BB               0
HP               0
KK               0
GD               0
PA               0
SB               0
CS               0
SF               0
BAT_ORDER        0
stadium_MH       0
stadium_MS       0
stadium_JS       0
stadium_SW       0
stadium_DJ       0
stadium_DG       0
stadium_GJ       0
stadium_GC       0
stadium_SJ       0
stadium_PH       0
stadium_US       0
stadium_CJ       0
stadium_CW       0
VS_T_ID_HH       0
VS_T_ID_HT       0
VS_T_ID_KT       0
VS_T_ID_LG       0
VS_T_ID_LT       0
VS_T_ID_NC       0
VS_T_ID_OB       0
VS_T_ID_SK       0
VS_T_ID_SS       0
VS_T_ID_WO       0
HOME             0
AWAY             0
inplay_ratio    77
P_ID             0
dtype: int64

In [34]:
df1 = df1.fillna(0)

In [35]:
df1['P_ID']=df1['P_ID'].astype('int')

In [36]:
batter_left

,year,month,P_ID,PA,T_ID,BABIP,BABIP_2020,total_BABIP
0,2020,10,65304,3.0,WO,0.375000,1.000000,0.875000
1,2020,10,64012,10.0,KT,0.297619,0.227273,0.241342
2,2020,10,68204,4.0,OB,0.315789,0.315789,0.315789
3,2020,10,66203,6.0,OB,0.285714,0.352941,0.339496
4,2020,10,50293,2.0,OB,0.266667,0.266667,0.266667
...,...,...,...,...,...,...,...,...
222,2020,10,63435,4.0,SS,0.166667,0.166667,0.166667
223,2020,10,69418,0.0,SS,0.299363,0.300699,0.300432
224,2020,10,61457,0.0,SS,0.000000,0.000000,0.000000
225,2020,10,66707,7.0,HH,0.272727,0.285714,0.283117


In [38]:
data = batter_tidy.loc[(batter_tidy['year']==2020) & (batter_tidy['month']==9)]

In [39]:
data

,year,month,P_ID,T_ID,AB,HIT,H2,H3,HR,BB,...,VS_T_ID_KT,VS_T_ID_LG,VS_T_ID_LT,VS_T_ID_NC,VS_T_ID_OB,VS_T_ID_SK,VS_T_ID_SS,VS_T_ID_WO,HOME,AWAY
7197,2020,9,65304,WO,2,2,0,0,0,0,...,2,2,2,2,5,4,2,0,14,13
7198,2020,9,64012,KT,7,4,1,0,1,2,...,0,2,3,2,4,4,4,2,13,12
7199,2020,9,68204,OB,4,2,1,0,0,0,...,4,2,0,2,0,2,4,5,14,11
7200,2020,9,66203,OB,6,3,1,0,0,0,...,4,2,0,2,0,2,4,5,14,11
7201,2020,9,50293,OB,2,1,0,0,0,0,...,4,2,0,2,0,2,4,5,14,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7419,2020,9,63435,SS,4,0,0,0,0,0,...,4,2,2,3,4,2,0,2,13,11
7420,2020,9,69418,SS,0,0,0,0,0,0,...,4,2,2,3,4,2,0,2,13,11
7421,2020,9,61457,SS,0,0,0,0,0,0,...,4,2,2,3,4,2,0,2,13,11
7422,2020,9,66707,HH,7,0,0,0,0,0,...,2,2,2,2,5,2,2,4,16,9


In [40]:
p_id = data['P_ID'].reset_index(drop = True)

In [41]:
batter_2020 = pd.DataFrame()

for i in range(len(p_id)):
    pid = p_id[i]
    batter = df1.loc[df1.P_ID == pid,:]
    batter_2020 = batter_2020.append(batter)
    

In [42]:
batter_2020 = batter_2020.reset_index(drop = True)

In [43]:
batter_2020

,year,month,AB,HIT,H2,H3,HR,BB,HP,KK,...,VS_T_ID_LT,VS_T_ID_NC,VS_T_ID_OB,VS_T_ID_SK,VS_T_ID_SS,VS_T_ID_WO,HOME,AWAY,inplay_ratio,P_ID
0,4040,17,2,2,0,0,0,0,0,0,...,7,4,5,4,6,0,29,24,0.666667,65304
1,10100,35,33,6,1,0,1,4,0,12,...,12,15,12,12,13,13,57,58,0.550000,64012
2,8080,30,47,12,1,0,0,3,0,13,...,8,13,0,9,13,11,50,45,0.703704,68204
3,6060,23,22,6,2,0,0,2,0,6,...,6,12,0,6,13,6,35,37,0.680000,66203
4,4040,15,17,4,0,0,0,3,1,2,...,0,8,0,4,7,6,23,25,0.714286,50293
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,4040,14,8,1,0,0,0,0,0,2,...,5,9,7,2,0,5,28,20,0.750000,63435
223,10100,35,168,44,11,0,1,7,2,28,...,13,10,16,12,0,16,62,56,0.790055,69418
224,2020,9,0,0,0,0,0,0,0,0,...,2,3,4,2,0,2,13,11,0.000000,61457
225,8080,30,34,6,2,0,0,0,1,13,...,10,12,10,10,11,10,50,44,0.600000,66707


In [44]:
batter_left= pd.merge(batter_left, df1[['P_ID','inplay_ratio']], left_on='P_ID', right_on='P_ID', how='left')

In [45]:
batter_left.index = batter_left['P_ID']
df1 = df1.reset_index(drop = True)

In [46]:
batter_left = batter_left.reset_index(drop = True)

In [47]:
batter_2020['inplay'] = batter_2020['AB'] - batter_2020['KK'] - batter_2020['HR'] + batter_2020['SF']

In [48]:
batter_2020['inplay_hit'] = batter_2020['HIT'] - batter_2020['HR']

In [49]:
batter_left= pd.merge(batter_left, batter_2020[['P_ID', 'inplay', 'inplay_hit']], left_on='P_ID', right_on='P_ID', how='left')

In [50]:
batter_left

,year,month,P_ID,PA,T_ID,BABIP,BABIP_2020,total_BABIP,inplay_ratio,inplay,inplay_hit
0,2020,10,65304,3.0,WO,0.375000,1.000000,0.875000,0.666667,2.0,2
1,2020,10,64012,10.0,KT,0.297619,0.227273,0.241342,0.550000,22.0,5
2,2020,10,68204,4.0,OB,0.315789,0.315789,0.315789,0.703704,38.0,12
3,2020,10,66203,6.0,OB,0.285714,0.352941,0.339496,0.680000,17.0,6
4,2020,10,50293,2.0,OB,0.266667,0.266667,0.266667,0.714286,15.0,4
...,...,...,...,...,...,...,...,...,...,...,...
222,2020,10,63435,4.0,SS,0.166667,0.166667,0.166667,0.750000,6.0,1
223,2020,10,69418,0.0,SS,0.299363,0.300699,0.300432,0.790055,143.0,43
224,2020,10,61457,0.0,SS,0.000000,0.000000,0.000000,0.000000,0.0,0
225,2020,10,66707,7.0,HH,0.272727,0.285714,0.283117,0.600000,21.0,6


In [51]:
batter_left['pred_inplay'] = batter_left['PA'] * batter_left['inplay_ratio']

In [52]:
batter_left

,year,month,P_ID,PA,T_ID,BABIP,BABIP_2020,total_BABIP,inplay_ratio,inplay,inplay_hit,pred_inplay
0,2020,10,65304,3.0,WO,0.375000,1.000000,0.875000,0.666667,2.0,2,2.000000
1,2020,10,64012,10.0,KT,0.297619,0.227273,0.241342,0.550000,22.0,5,5.500000
2,2020,10,68204,4.0,OB,0.315789,0.315789,0.315789,0.703704,38.0,12,2.814815
3,2020,10,66203,6.0,OB,0.285714,0.352941,0.339496,0.680000,17.0,6,4.080000
4,2020,10,50293,2.0,OB,0.266667,0.266667,0.266667,0.714286,15.0,4,1.428571
...,...,...,...,...,...,...,...,...,...,...,...,...
222,2020,10,63435,4.0,SS,0.166667,0.166667,0.166667,0.750000,6.0,1,3.000000
223,2020,10,69418,0.0,SS,0.299363,0.300699,0.300432,0.790055,143.0,43,0.000000
224,2020,10,61457,0.0,SS,0.000000,0.000000,0.000000,0.000000,0.0,0,0.000000
225,2020,10,66707,7.0,HH,0.272727,0.285714,0.283117,0.600000,21.0,6,4.200000


In [53]:
batter_left['pred_inplay_hit'] = batter_left['total_BABIP'] * (batter_left['inplay'] + batter_left['pred_inplay']) - batter_left['inplay_hit']

In [54]:
batter_left = batter_left[['year', 'month', 'P_ID', 'T_ID', 'PA', 'pred_inplay_hit']]

In [55]:
batter_left = batter_left.fillna(0)

In [56]:
batter_left.pred_inplay_hit[batter_left.pred_inplay_hit < 0.0 ]=0

C:\Users\dhxog\anaconda3\envs\my\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [57]:
batter_left

,year,month,P_ID,T_ID,PA,pred_inplay_hit
0,2020,10,65304,WO,3.0,1.500000
1,2020,10,64012,KT,10.0,1.636905
2,2020,10,68204,OB,4.0,0.888889
3,2020,10,66203,OB,6.0,1.156571
4,2020,10,50293,OB,2.0,0.380952
...,...,...,...,...,...,...
222,2020,10,63435,SS,4.0,0.500000
223,2020,10,69418,SS,0.0,0.000000
224,2020,10,61457,SS,0.0,0.000000
225,2020,10,66707,HH,7.0,1.134545


In [157]:
batter_2020_ratio = pd.read_csv("C:/Users/njj06/Desktop/ESC_summer/Baseball_ChilliShrimp/data/batter_2020_ratio.csv")

In [158]:
batter_2020_ratio

,P_ID,H2_rate,H3_rate,HR_rate,BB_rate,HP_rate,KK_rate,BK_rate
0,50030,0.166667,0.000000,0.055556,0.075107,0.010730,0.141631,0.000000
1,50036,0.111111,0.111111,0.111111,0.151515,0.090909,0.090909,0.000000
2,50040,0.141243,0.005650,0.079096,0.073713,0.009736,0.173853,0.000000
3,50109,0.166667,0.000000,0.000000,0.050000,0.050000,0.050000,0.000000
4,50126,0.219178,0.000000,0.054795,0.084795,0.026316,0.143275,0.002924
5,50157,0.172414,0.000000,0.103448,0.076233,0.026906,0.147982,0.000000
6,50203,0.083333,0.000000,0.083333,0.048780,0.048780,0.219512,0.000000
7,50234,0.152778,0.000000,0.069444,0.081818,0.003030,0.251515,0.000000
8,50262,0.333333,0.000000,0.000000,0.368421,0.000000,0.105263,0.000000
9,50296,0.166667,0.000000,0.166667,0.225806,0.032258,0.193548,0.000000


In [159]:
batter_left = pd.merge(batter_left, batter_2020_ratio, left_on='P_ID', right_on='P_ID', how='left')

In [160]:
batter_left

,year,month,P_ID,T_ID,PA,pred_inplay_hit,H2_rate,H3_rate,HR_rate,BB_rate,HP_rate,KK_rate,BK_rate
0,2020,10,65304,WO,3.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020,10,64012,KT,10.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020,10,68204,OB,4.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020,10,66203,OB,6.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020,10,50293,OB,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2020,10,64213,OB,8.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2020,10,62356,WO,3.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2020,10,79192,LG,22.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2020,10,64469,SS,3.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2020,10,60343,HH,28.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
